In [1]:
import openrouteservice as ors
import folium
import folium.plugins as plugins
import random
import pandas as pd

client = ors.Client(key='5b3ce3597851110001cf6248facc383c642b420fa9818331b313169e')

In [2]:
def visualize(readLocation, writeFolder, writeLocation):
    # Read Data
    nodes = pd.read_csv(readLocation + "\\Nodes.csv")
    truckRoute = pd.read_csv(readLocation + "\\TruckRoute.csv")
    droneRoute = pd.read_csv(readLocation + "\\DroneRoute.csv")
    
    m = folium.Map(location=[40.80533085, 29.442315], tiles='cartodbpositron', zoom_start=13)

    # Add Nodes
    for index, row in nodes.iterrows():
        color = None
        if(row["Type"] == "UAVELIGIBLE"):
            color = "orange"
        else:
            color = "cyan"
        if(index == 0 or index == nodes.shape[0]-1):
            icon = plugins.BeautifyIcon(icon = "truck", iconShape="marker", backgroundColor="white")
        else:
            icon = plugins.BeautifyIcon(icon = "", iconShape="marker", number = row["Index"], backgroundColor=color)
        folium.Marker([row["Y"], row["X"]], icon=icon, popup= row["Index"],).add_to(m)
        
    # Add Truck Routes
    for i in range(0, truckRoute.shape[0] - 1):
        popup = "{i} --> {j}"

        coordinates = [[truckRoute["X"][i], truckRoute["Y"][i]], [truckRoute["X"][i+1], truckRoute["Y"][i+1]]]
        route = client.directions(
        coordinates=coordinates,
        profile='driving-car',
        format='geojson',
        validate=False,)
        plugins.AntPath(locations=[list(reversed(coord)) 
                               for coord in 
                               route['features'][0]['geometry']['coordinates']], 
                               tooltip = popup.format(i = truckRoute["Index"][i], j = truckRoute["Index"][i+1]), linecap ="round").add_to(m)

    # Add Drone Routes
    colorsForDroneRoutes = ["red", "purple", "yellow", "green", "orange"]

    for rowNumber in range(0, droneRoute.shape[0]):
        i = (droneRoute["iY"][rowNumber], droneRoute["iX"][rowNumber])
        j = (droneRoute["jY"][rowNumber], droneRoute["jX"][rowNumber])
        k = (droneRoute["kY"][rowNumber], droneRoute["kX"][rowNumber])

        color = colorsForDroneRoutes[droneRoute["DroneIndex"][rowNumber]]

        ijRouteToolTip = "{i} --> {j} | LaunchTime[{i}]: {launch} / ArrivalTime[{j}]: {arrival}"
        jkRouteToolTip = "{j} --> {k} | LaunchTime[{j}]: {launch} / RetrievalTime[{k}]: {retrieval}"
        plugins.AntPath(locations = [i,j], color=color, tooltip = ijRouteToolTip.format(i = droneRoute["iIndex"][rowNumber],
                                                                                       j = droneRoute["jIndex"][rowNumber],
                                                                                       launch = droneRoute["Launch[i]"][rowNumber],
                                                                                       arrival = droneRoute["Arrival[j]"][rowNumber])).add_to(m)
        plugins.AntPath(locations = [j,k], color=color, tooltip = jkRouteToolTip.format(j = droneRoute["jIndex"][rowNumber],
                                                                                       k = droneRoute["kIndex"][rowNumber],
                                                                                       launch = droneRoute["Launch[j]"][rowNumber],
                                                                                       retrieval = droneRoute["Retrieval[k]"][rowNumber])).add_to(m)
    #Save visualization
    m.save(writeLocation)

In [3]:
nArray = [10, 20]
iTimes = [1, 2]
drones = [1, 2]

for n in nArray:
    for i in iTimes:
        for d in drones:
            readLocation = "SolutionData\\Data_{n}_{d}_{i}"
            readLocation = readLocation.format(n = n, d = d,i = i)

            writeFolder = "SolutionData\\Data_{n}_{d}_{i}"
            writeFolder = writeFolder.format(n = n, d = d,i = i)

            writeLocation = "SolutionData\\Data_{n}_{d}_{i}\\map.html"
            writeLocation = writeLocation.format(n = n, d = d,i = i)

            visualize(readLocation, writeFolder, writeLocation)


FileNotFoundError: [Errno 2] No such file or directory: 'SolutionData\\Data_20_1_1\\TruckRoute.csv'